### GH Access Token

In [19]:
def load_token(filename):
    with open(filename, 'r') as f:
        token = f.read().strip()
    return token

GH_ACCESS_TOKEN = None #load_token("gh_access_token.txt")

1. Log in to your GitHub account.
2. Click on your profile picture in the top-right corner of the screen and select "Settings".
3. In the left sidebar, click on "Developer settings".
4. Click on "Personal access tokens".
5. Click on "Generate new token".
6. Give your token a description that will help you remember what it is used for.
7. Select the permissions that your token requires. Note that you should only select the permissions that are necessary for your application to function.
8. Click on "Generate token".
9. Copy the generated access token.
10. Open a text editor and create a new file called gh_access_token.txt.
11. Paste the access token into the file.
12. Save and close the file.


### Regular Imports

In [20]:
import sys
import ast
import git
import numpy as np
import random
from tqdm.auto import tqdm, trange
import os
from datetime import datetime
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor, as_completed, wait, ALL_COMPLETED
import networkx as nx
import tree_sitter
from tree_sitter import Language
import json
import requests
import pandas as pd
from collections import defaultdict
from github import Github
import base64
import cProfile
import pstats


In [21]:
#Disable import warnings for tensorflow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['OMP_NUM_THREADS'] = '3'
os.environ["TOKENIZERS_PARALLELISM"] = "false"

### Machine Learning Imports

In [22]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import (
    Input, Dense, LSTM, Attention, Concatenate, Layer,
    Embedding, Dot, Softmax, TimeDistributed, Multiply,
    Lambda, LayerNormalization, MultiHeadAttention,
    Add, Masking, GlobalMaxPooling1D, GlobalMaxPooling2D, Reshape, MaxPooling1D, MaxPooling2D,
    Dropout, Conv1D, Conv2D, Bidirectional, GRU, ConvLSTM2D, Flatten, Permute
)

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import LambdaCallback, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MultiLabelBinarizer
from sklearn.cluster import KMeans

from tensorflow.keras.models import Model
from tensorflow.keras.losses import MeanSquaredError, CosineSimilarity
from tensorflow.keras import backend as K
from keras.utils.vis_utils import plot_model
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans

### Data Pre-Processing Parameters
These must be set before running the data preprocessing
If any of these parameters are changed, all cells below **must** be re-run

In [23]:
#Number of contexts to pad/reduce to per sample
#This is the `k` number described in commit2seq
BAG_SIZE = 32 # 256

#Maximum depth of context
#This is the depth of the path itself, so with a depth of 32, paths can be a maximum of 32 notes deep.
#Smaller paths mean a higher focus on local dependencies, while longer paths represent a more general
#representation of long-distance dependencies.
CONTEXT_SIZE = 8 # 16

#Internal fixed length representation size. This is the size of the fixed-length vector that the encoder
#will learn to produce. Experimentation will be needed.
OUTPUT_SIZE = 256 # 2048

#Toggle to determine if tokens should be one-hot encoded or not
ONE_HOT = False

#Toogle to use root -> terminal paths instead of terminal -> terminal paths. 
#code2vec and commit2vec use terminal -> terminal paths, but generating them is significantly slower.
ONLY_ROOT_PATHS = False

#Toggle to drop paths above the context size limit. Likely should always be set to true.
IGNORE_DEEP_PATHS = True

#Generate full tuples of (terminal_token_a, (path...), terminal_token_b) instead of just paths.
#commit2seq describes using this strategy as being quite a bit more effective (like 30%), but currently I
#haven't figured out how to implement it.
USE_FULL_TUPLES = False    #NOT working yet

### Token Vocab and Meta Setup

In [24]:
#Load language files
#The ast.so file must be re-generated to add support for more languages.
#To do this, see: https://github.com/tree-sitter/py-tree-sitter

JS_LANGUAGE = Language('ast-bindings/build/ast.so', 'javascript')
PY_LANGUAGE = Language('ast-bindings/build/ast.so', 'python')
JAVA_LANGUAGE = Language('ast-bindings/build/ast.so', 'java')
C_LANGUAGE = Language('ast-bindings/build/ast.so', 'c')
CPP_LANGUAGE = Language('ast-bindings/build/ast.so', 'cpp')

In [25]:
def download_json_data(lang):
    url = f"https://raw.githubusercontent.com/tree-sitter/tree-sitter-{lang}/master/src/node-types.json"
    data = json.loads(requests.get(url).text)
    types = [node_type["type"] for node_type in data]
    for node_type in data:
        if "subtypes" in node_type:
            subtypes = [subtype["type"] for subtype in node_type["subtypes"]]
            types.extend(subtypes)
    types = list(set(types))
    return types

PYTHON_NODE_TYPES = download_json_data("python")
JAVA_NODE_TYPES = download_json_data("java")
JAVASCRIPT_NODE_TYPES = download_json_data("javascript")
C_NODE_TYPES = download_json_data("c")
CPP_NODE_TYPES = download_json_data("cpp")

In [26]:
#For now set to just use JAVA node types. To enable support for other languages, just use a set union.
ALL_NODE_TYPES = JAVA_NODE_TYPES
#ALL_NODE_TYPES = list(set(python_data + java_data + javascript_data + c_data + cpp_data))

# FILE_FILTERS = (".c", ".cpp", ".java", ".js", ".py")
FILE_FILTERS = (".java")

MAX_NODE_LOOKUP_NUM = len(ALL_NODE_TYPES)
ALL_NODE_INDEXES = range(MAX_NODE_LOOKUP_NUM)

In [27]:
#Automatically generated INPUT_SHAPE param. Don't modify
if(ONE_HOT):
    INPUT_SHAPE = (BAG_SIZE, CONTEXT_SIZE * (MAX_NODE_LOOKUP_NUM + 1))
else:
    INPUT_SHAPE = (BAG_SIZE, CONTEXT_SIZE)

### Load Raw dataset

In [28]:
RAW_DATA = pd.read_csv("pairs_apache_small.csv")

In [29]:
RAW_DATA

,fix_hash,bug_hash,Y,bug_repo,fix_repo
0,0b05d3b939c5ed37a4253e7c3614d824e76ed664,0b92cec1e07a1f2d9aa70f3ecd7d0fb12290d2e2,1,apache/kafka,apache/kafka
1,f84bf9cbe771f252d5624e30d27755c9e5225179,d46e5634a3bca248e00b4f44e5216cd6607c5a52,1,apache/hbase,apache/hbase
2,6c8f14b78afc0a6721433af6554b0ab45b8e163d,53989648276fc057d5ec7ab056a7d0a654d110b8,1,apache/hive,apache/hive
3,9623ae4de3e4c65ab2e90798843769795b8a00c6,505cf9d618c24f532e7b800d8b34a20a40c45feb,1,apache/kafka,apache/kafka
4,9fe8802cb83e05c0392b11b8dcfe354fecfda786,2e5c28f8c927f2caf6dcffc9070671089c5f771f,1,apache/hive,apache/hive
...,...,...,...,...,...
877,16c934812b2395da7fb3968965b03d2f6aa8c8a3,0a08525ad236f78df05c854dead62f300eae271d,0,apache/cassandra,apache/cassandra
878,b04bed022aec0f1f478a03383ab5184f048133b6,44f6c4b946511ce4663d41bf40f2960d2faee198,0,apache/kafka,apache/kafka
879,b04bed022aec0f1f478a03383ab5184f048133b6,0d4cf64af359d22749f7e865c4efaee773d64962,0,apache/kafka,apache/kafka
880,925599c72bc35edb5ffa25c6cc839932573b01aa,a8a73362bd9a3967c46011ded1ed831a586acd2e,0,apache/zookeeper,apache/zookeeper


### Load Rich Dataset from GH/git

In [30]:
REPO_LOOKUP = defaultdict(list)

repos = set(list(RAW_DATA["bug_repo"]) + list(RAW_DATA["fix_repo"]))
#Create a "clones" directory in order to clone local repos
if not os.path.exists("clones"):
    os.makedirs("clones")

# Clone each "repo" into the "clones" folder
for repo in tqdm(repos):
    repo_folder = repo.replace("/", "-")
    if not os.path.exists(f"clones/{repo_folder}"):
        os.system(f"git clone https://github.com/{repo}.git clones/{repo_folder}")
    
    #TODO: Try to implement this. Currently it breaks concurrant pre-processing
    #REPO_LOOKUP[f"clones/{repo_folder}"] = git.Repo(f"clones/{repo_folder}")

  0%|          | 0/13 [00:00<?, ?it/s]

#### Commit Helper Class

In [31]:
class Commit:
    
    ###################################### SETUP ###################################### 
    
    def __init__(self, sha, repo_path):
        self.sha = sha
        self.repo_path = repo_path
        self.parent = None
        self.author = None
        self.datetime = None
        self.message = None
        self.bag_of_contexts = None
        self._populate_commit_info()

    def _populate_commit_info(self):
        if(self.repo_path in REPO_LOOKUP):
            repo = REPO_LOOKUP[self.repo_path]
        else:
            repo = git.Repo(self.repo_path)
        commit = repo.commit(self.sha)
        self.parent = commit.parents[0].hexsha if commit.parents else None
        self.author = commit.author.name if commit.author else None
        self.datetime = datetime.fromtimestamp(commit.committed_date)
        self.message = commit.message if commit.message else None
    
    def _GH_populate_commit_info(self):
        g = Github(GH_ACCESS_TOKEN)
        repo = g.get_repo(self.repo_name)
        commit = repo.get_commit(sha=self.sha)
        self.parent = commit.parents[0].sha if commit.parents else None
        self.author = commit.author.name if commit.author else None
        self.datetime = commit.commit.author.date
        self.message = commit.commit.message if commit.commit.message else None
        
    def _generate_bags_of_contexts(self):
        self.bag_of_contexts = self.to_padded_consumable_data()

    ###################################### GIT and GitHub ###################################### 
        
    ################# GIT #################
    
    def get_files_changed(self):
        try:
            if(self.repo_path in REPO_LOOKUP):
                repo = REPO_LOOKUP[self.repo_path]
            else:
                repo = git.Repo(self.repo_path)
            commit = repo.commit(self.sha)
            return [diff.a_path for diff in commit.diff(commit.parents[0]) if diff.a_path.endswith(FILE_FILTERS)]
        except:
            return []

    def get_source_at_commit(self, file_name):
        try:
            if(self.repo_path in REPO_LOOKUP):
                repo = REPO_LOOKUP[self.repo_path]
            else:
                repo = git.Repo(self.repo_path)
            commit = repo.commit(self.sha)
            return commit.tree[file_name].data_stream.read().decode('utf-8')
        except:
            return ''

    def get_source_at_parent(self, file_name):
        try:
            if(self.repo_path in REPO_LOOKUP):
                repo = REPO_LOOKUP[self.repo_path]
            else:
                repo = git.Repo(self.repo_path)
            commit = repo.commit(self.sha)
            return commit.parents[0].tree[file_name].data_stream.read().decode('utf-8')
        except:
            return ''
    
    ################# GitHub #################
    #### Still todo, in testing ####
    
    def gh_get_files_changed(self):
        try:
            g = Github(GH_ACCESS_TOKEN)
            repo = g.get_repo(self.repo_name)
            commit = repo.get_commit(sha=self.sha)
            return [f.filename for f in commit.files if f.filename.endswith(FILE_FILTERS)]
        except:
            return []

    def gh_get_source_at_commit(self, file_name):
        try:
            g = Github(GH_ACCESS_TOKEN)
            repo = g.get_repo(self.repo_name)
            contents = repo.get_contents(file_name, ref=self.sha)
            return base64.b64decode(contents.content).decode('utf-8')
        except:
            return ''

    def gh_get_source_at_parent(self, file_name):
        try:
            g = Github(GH_ACCESS_TOKEN)
            repo = g.get_repo(self.repo_name)
            commit = repo.get_commit(sha=self.sha)
            parent_commit = repo.get_commit(sha=commit.parents[0].sha)
            contents = repo.get_contents(file_name, ref=parent_commit.sha)
            return base64.b64decode(contents.content).decode('utf-8')
        except:
            return ''

    ###################################### AST Processing ###################################### 

    def source_to_ast(self, source, file_name):
        try:
            parser = tree_sitter.Parser()
            if file_name.endswith('.c'):
                parser.set_language(C_LANGUAGE)
            elif file_name.endswith('.cpp'):
                parser.set_language(CPP_LANGUAGE)
            elif file_name.endswith('.java'):
                parser.set_language(JAVA_LANGUAGE)
            elif file_name.endswith('.js'):
                parser.set_language(JS_LANGUAGE)
            elif file_name.endswith('.py'):
                parser.set_language(PY_LANGUAGE)
            else:
                print("UNKNOWN LANGUAGE")
                return None
            return parser.parse(bytes(source, 'utf8'))
        except:
            return None
        
    ###################################### Bag of Contexts Processing ###################################### 
        
    def get_root_paths(self, tree):
        try:
            paths = set()

            # Recursive function to explore the tree
            def explore(node, path, terminalA=None):
                # Add current node to path
                if terminalA is None:
                    terminalA = node
                path.append(node.type)

                # If the node has no children, it's a leaf node and the path is complete
                if not node.child_count:
                    if USE_FULL_TUPLES:
                        paths.add((terminalA.text, tuple(path), node.text))
                    else:
                        paths.add(tuple(path))
                else:
                    # Explore each child node recursively
                    for child in node.children:
                        explore(child, path, terminalA)

                # Remove current node from path before returning
                path.pop()

            # Start exploring from the root node
            root_node = tree.root_node
            explore(root_node, [])
            
            return paths
        except:
            return set([])

    def get_paths(self, ast):
        graph = nx.Graph()
        node_id = 0
        leaf_nodes = []

        def add_node(node):
            nonlocal node_id
            node_name = node.type
            node_id += 1
            graph.add_node(node_id, name=node_name, node=node, text=node.text)
            if not node.children:
                leaf_nodes.append(node_id)
            return node_id

        def add_edge(parent_id, child_id):
            graph.add_edge(parent_id, child_id)

        def traverse(node, parent_id=None):
            current_id = add_node(node)
            if parent_id is not None:
                add_edge(parent_id, current_id)
            for child in node.children:
                traverse(child, current_id)

        traverse(ast.root_node)

        if BAG_SIZE != None:
            leaf_nodes_sample = random.sample(leaf_nodes, min(BAG_SIZE, len(leaf_nodes)))
        else:
            leaf_nodes_sample = leaf_nodes
        leaf_node_pairs = [(leaf_nodes_sample[i], leaf_nodes_sample[j]) for i in range(len(leaf_nodes_sample)) for j in range(i+1, len(leaf_nodes_sample))]
        
        if BAG_SIZE != None:
            leaf_node_pairs = random.sample(leaf_node_pairs, min(BAG_SIZE,len(leaf_node_pairs)))
        
        all_paths = []
        for pair in leaf_node_pairs:
            path = list(nx.all_simple_paths(graph, source=pair[0], target=pair[1]))[0]
            node_types = [graph.nodes[nodeID]['name'] for nodeID in path]
            if(USE_FULL_TUPLES):
                all_paths.append((graph.nodes[pair[0]]['text'], tuple(node_types), graph.nodes[pair[1]]['text']))
            else:
                all_paths.append(tuple(node_types))
                
        return set(all_paths)


    def ast_to_bag_of_contexts(self, ast_trees):
        paths = set()
        for tree in ast_trees:
            if(ONLY_ROOT_PATHS):
                paths |= self.get_root_paths(tree)
            else:
                paths |= self.get_paths(tree)
        return paths

    ###################################### Padding and Encoding ###################################### 
    
    def map_bag_of_contexts_to_id(self, bag_of_contexts):
        mapped_paths = []
        for path in bag_of_contexts:
            mapped_path = []
            for node in path:
                index = ALL_NODE_TYPES.index(node)
                mapped_path.append(index + 1)
            mapped_paths.append(mapped_path)
        return mapped_paths

    def one_hot_encode(self, bag_of_contexts):
        one_hot_paths = []

        # Iterate over each row in the array
        for row in bag_of_contexts:
            # Create an empty list to hold the one-hot encodings for this row
            row_one_hot = []

            # Iterate over each element in the row
            for num in row:
                # Create an empty list to hold the one-hot encoding for this number
                num_one_hot = [0] * (MAX_NODE_LOOKUP_NUM + 1)

                # Set the corresponding element to 1
                num_one_hot[int(num)] = 1

                # Add the one-hot encoding for this number to the row's list
                row_one_hot.append(num_one_hot)

            # Add the row's list of one-hot encodings to the main list
            one_hot_paths.append(row_one_hot)

        return one_hot_paths

    def pad_each_context(self, bag_of_contexts):
        padded_one_hot_paths = []
        for path in bag_of_contexts:
            if IGNORE_DEEP_PATHS and len(path) > CONTEXT_SIZE:
                continue
            if ONE_HOT:
                padded_path = [[0] * (MAX_NODE_LOOKUP_NUM + 1)] * max(CONTEXT_SIZE - len(path), 0) + path[-CONTEXT_SIZE:]
            else:
                padded_path = [0] * max(CONTEXT_SIZE - len(path), 0) + path[-CONTEXT_SIZE:]
            padded_one_hot_paths.append(padded_path)
        return padded_one_hot_paths

    ###################################### Utility ###################################### 
    
    def to_raw_consumable_data(self):
        files_changed = self.get_files_changed()[:32]
        sources_at_commit = [self.get_source_at_commit(filename) for filename in files_changed]
        sources_at_parent = [self.get_source_at_parent(filename) for filename in files_changed]        

        asts_commit = [self.source_to_ast(source, files_changed[i]) for i, source in enumerate(sources_at_commit)]
        asts_parent = [self.source_to_ast(source, files_changed[i]) for i, source in enumerate(sources_at_parent)]
        
        contexts_commit = self.ast_to_bag_of_contexts(asts_commit)
        contexts_parent = self.ast_to_bag_of_contexts(asts_parent)

        contexts = list(contexts_commit.symmetric_difference(contexts_parent))

        
        contexts = self.map_bag_of_contexts_to_id(contexts)

        if(ONE_HOT):
            contexts = self.one_hot_encode(contexts)

        contexts = self.pad_each_context(contexts)

        return contexts
    
    def raw_to_padded(self, consumable):
        if(len(consumable) == BAG_SIZE):
            return consumable
        
        if(len(consumable) > BAG_SIZE):
            return random.sample(consumable, BAG_SIZE)
        
        if ONE_HOT:
            blank_path = [[0] * (MAX_NODE_LOOKUP_NUM + 1)] * CONTEXT_SIZE
        else:
            blank_path = ([0] * CONTEXT_SIZE)
            
        return ([blank_path] * (BAG_SIZE - len(consumable)) + consumable)

    
    def to_padded_consumable_data(self):
        consumable = self.to_raw_consumable_data()
        padded = self.raw_to_padded(consumable)
        if(ONE_HOT):
            padded = np.array(padded)
            padded = padded.reshape(padded.shape[0], padded.shape[1] * padded.shape[2])
            padded = padded.tolist()
        return padded

#### Data Extraction

In [32]:
RAW_DATA = pd.concat([RAW_DATA.iloc[:32], RAW_DATA.iloc[-32:]])
TUPLES = [(row[0], row[4]) for i, row in RAW_DATA.iterrows()] + [(row[1], row[3]) for i, row in RAW_DATA.iterrows()]
TUPLES = list(set(TUPLES))

#### Note: This takes a long time

In [33]:
#Creates a lookup dictionary where any commit SHA can be looked up to grab the Commit object with all the data, + bag of paths
COMMIT_DATA_LOOKUP = defaultdict(list)

def _to_commit(pair):
    sha = pair[0]
    repo = pair[1]
    commit = Commit(sha, f"clones/{repo.replace('/', '-')}")
    commit._populate_commit_info()
    commit._generate_bags_of_contexts()
    
    return commit

resolved = []
with ThreadPoolExecutor(max_workers=16) as executor:
    futures = [executor.submit(_to_commit, pair) for pair in TUPLES]
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing rich data from commits"):
        try:
            resolved.append(future.result())
        except Exception as e:
            resolved.append(None)
            print(e)

for commit in tqdm(resolved, desc="Adding to lookup dictionary"):
    if commit is not None:
        COMMIT_DATA_LOOKUP[commit.sha] = commit

Processing rich data from ommits:   0%|          | 0/110 [00:00<?, ?it/s]

SHA b'eb91c4c03e8400811f7291294b6132f9a22a4d3f' could not be resolved, git returned: b'eb91c4c03e8400811f7291294b6132f9a22a4d3f missing'
SHA b'a8abd6b023e98903e9f200c36173f23a8022f907' could not be resolved, git returned: b'a8abd6b023e98903e9f200c36173f23a8022f907 missing'


Adding to lookup dictionary:   0%|          | 0/110 [00:00<?, ?it/s]

### Metadata processing

In [49]:
def processNames(name1: str, name2: str):
    if name1 == name2:
        return 1
    else:
        return 0

# Process two timestamnps and return their positive difference
def processTimestamps(timestamp1: int, timestamp2: int):
    difference = timestamp1 - timestamp2
    if(difference < 0):
        difference *= -1
    return difference

# Process two commit messages and return a similarity score from 0 to 1
def processCommitMessages(commit1: str, commit2: str):
    similarity = cosineSimilarity(commit1, commit2)
    return similarity


# Calculate the cosine similarity of two strings
def cosineSimilarity(str1: str, str2: str):
    vectors = sentences2Vecs([str1, str2])
    similarity = cosine_similarity(vectors[0].reshape(1, -1), vectors[1].reshape(1,-1))[0][0]
    normalized_similarity = round((similarity + 1) / 2, 6)
    return normalized_similarity

# Turn an array of sentences into an array of vectors using a special sentence transformer using their paraphrase-MiniLM-L6-v2 model for relatively quick performance.  
# In my small amount of testing, takes about 4 seconds to load the model the first time and perform the transformations, and then 0.3 secconds after that
def sentences2Vecs(sentences):
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    sentence_embeddings = model.encode(sentences)
    return sentence_embeddings

### Dataset finalization

In [63]:
X_train = [
    (
        np.array(processNames(COMMIT_DATA_LOOKUP[row[0]].author, COMMIT_DATA_LOOKUP[row[1]].author)),
        np.array(processTimestamps(COMMIT_DATA_LOOKUP[row[0]].datetime.timestamp(), COMMIT_DATA_LOOKUP[row[1]].datetime.timestamp())),
        np.array(processCommitMessages(COMMIT_DATA_LOOKUP[row[0]].message, COMMIT_DATA_LOOKUP[row[1]].message)),
        np.array(COMMIT_DATA_LOOKUP[row[0]].bag_of_contexts),
        np.array(COMMIT_DATA_LOOKUP[row[1]].bag_of_contexts)
    )
    for i, row in tqdm(RAW_DATA.iterrows(), total=len(RAW_DATA), desc="Generating X_train") if row[0] in COMMIT_DATA_LOOKUP and row[1] in COMMIT_DATA_LOOKUP
]

Generating X_train:   0%|          | 0/64 [00:00<?, ?it/s]

In [64]:
y_train = [row[2] for i, row in RAW_DATA.iterrows() if row[0] in COMMIT_DATA_LOOKUP and row[1] in COMMIT_DATA_LOOKUP]

In [65]:
ALL_BAGS_OF_CONTEXTS = np.array([COMMIT_DATA_LOOKUP[sha].bag_of_contexts for sha in COMMIT_DATA_LOOKUP])

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [67]:
np.sum(y_train) / len(y_train), len(y_train)

(0.5, 50)

### Model Setup

In [68]:
class CommitDiffModel:
    def __init__(self):
        self.input_shape = INPUT_SHAPE
        self.example_size = BAG_SIZE
        self.context_size = CONTEXT_SIZE
        self.fixed_vector_size = OUTPUT_SIZE
        self.num_heads = 4
        self.key_dim = 512
        self.units = 128
        self.rate = 0.1
        self.activation_fn1 = "relu"
        self.activation_fn2 = "relu"
        self.activation_fn3 = "relu"
        self.optimizer = "adam"
        self.loss_fn = "mse"
        self.temperature = 0.1
        self.encoder = None
        self.siam_model = None
        self.binary_classification_model = None

    def initialize(self):
        self.encoder = self.build_encoder()
        self.siam_model = self.build_siam_model()
        self.binary_classification_model = self.build_binary_classification_model()
        
    def encoder_attention_feed_forward(self, inputs):
        
        inputs = Reshape((self.example_size, -1))(inputs)
        
        # Self-attention layer to obtain attention weights
        self_attention = MultiHeadAttention(num_heads=self.num_heads, key_dim=self.key_dim)(inputs, inputs)
        
        normalized = LayerNormalization()(self_attention)
        
        # Feedforward layers
        feedforward = TimeDistributed(Dense(units=self.units, activation=self.activation_fn1))(normalized)

        # Global max pooling layer to produce a fixed-length output vector representation
        pooled = MaxPooling1D(pool_size=self.context_size)(feedforward)

        return pooled
    
    def encoder_attention_weighted(self, inputs):
        
        # Add a self-attention layer to compute attention weights for each context
        self_attention = MultiHeadAttention(num_heads=self.num_heads, key_dim=self.key_dim)(inputs, inputs)

        # Add a layer normalization layer to normalize the self-attention output
        normalized = LayerNormalization()(self_attention)

        # Add a feedforward layer to compute context-level encodings
        encoded = Dense(units=self.units, activation='relu')(normalized)

        # Compute attention weights based on the contents of each context
        attention_weights = Dense(units=1, activation='softmax')(encoded)

        # Compute a weighted sum of the contexts using attention weights
        weighted_sum = tf.keras.layers.Dot(axes=1)([attention_weights, encoded])
        
        return weighted_sum
    
    
    #NOTE: Conceptually good, but doesn't actually compile :(
    def encoder_softmax_pooled(self, inputs):
        
        # Self-attention layer to obtain attention weights
        self_attention = MultiHeadAttention(num_heads=self.num_heads, key_dim=self.key_dim)(inputs, inputs)

        # Compute attention weights and apply them to the inputs
        attention_weights = Softmax()(self_attention)
        weighted_inputs = Multiply()([attention_weights, inputs])

        # Sum pooling to aggregate the weighted bag
        pooled_inputs = Lambda(lambda x: K.sum(x, axis=1))(weighted_inputs)

        normalized = LayerNormalization()(pooled_inputs)
        
        # Feedforward layers
        feedforward = TimeDistributed(Dense(units=self.units, activation=self.activation_fn1))(normalized)

        # Global max pooling layer to produce a fixed-length output vector representation
        pooled = MaxPooling1D(pool_size=self.context_size)(feedforward)

        return pooled
    
    #Best so far. Haven't played with hyperparams
    def encoder_recurrent_convolutional(self, inputs):
       
        # Add a 1D convolutional layer to extract features from each context
        conv = Conv1D(filters=32, kernel_size=3, activation='relu')(inputs)

        # Add a max pooling layer to summarize the extracted features
        max_pooling = MaxPooling1D(pool_size=self.context_size)(conv)

        # Add a recurrent layer to capture temporal dependencies within each context
        lstm = LSTM(units=64)(max_pooling)

        return lstm
    
    def encoder_hierarchical(self, inputs):
        
        # Encode each individual context using a bidirectional LSTM layer
        encoded_contexts = Bidirectional(LSTM(units=self.units, return_sequences=True))(inputs)

        # Compute a weighted sum of the context-level encodings, using attention weights based on the contents of each context
        attention_weights = TimeDistributed(Dense(units=1, activation='softmax'))(encoded_contexts)
        weighted_sum = tf.keras.layers.Dot(axes=1)([attention_weights, encoded_contexts])

        return weighted_sum
        
    def build_encoder(self):
        
        inputs = Input(shape=self.input_shape)

        masked_inputs = Masking()(inputs)
        
        ##########################################################
        
        # encoded = self.encoder_attention_feed_forward(masked_inputs)
        # encoded = self.encoder_softmax_pooled(masked_inputs) #Broken
        encoded = self.encoder_recurrent_convolutional(masked_inputs) #Best so far with small inputs
        # encoded = self.encoder_attention_weighted(masked_inputs)
        # encoded = self.encoder_hierarchical(masked_inputs)
        
        ##########################################################

        # Dropout layer to prevent overfitting
        dropout = Dropout(rate=self.rate)(encoded)

        # Fixed length vector representation
        outputs = Dense(units=self.fixed_vector_size)(dropout)

        # Define encoder model
        encoder = tf.keras.Model(inputs=inputs, outputs=outputs)
        return encoder
    
    def build_siam_model(self):
        
        # Create SimSiam model
        x1 = Input(shape=self.input_shape)
        x2 = Input(shape=self.input_shape)
        
        x1 = tf.random.shuffle(x1)
        x2 = tf.random.shuffle(x2)
        
        # Encode the input twice using the same encoder
        z1 = self.encoder(x1)
        z2 = self.encoder(x2)
        
        # Predict a transformation of the first encoding
        p1 = Dense(units=self.fixed_vector_size, activation=self.activation_fn2)(z1)
        p2 = Dense(units=self.fixed_vector_size, activation=self.activation_fn2)(z1)
        
        #Loss function
        def D(p, z):
            z = tf.stop_gradient(z)
            p = tf.math.l2_normalize(p, axis=1)
            z = tf.math.l2_normalize(z, axis=1)
            return -tf.reduce_mean(tf.reduce_sum(p * z, axis=1))
        
        loss = D(p1, z2) / 2 + D(p2, z1) / 2
        
        # Define the model
        model = tf.keras.Model(inputs=[x1,x2], outputs=loss)
        
        # Compile the model
        model.compile(optimizer=self.optimizer, loss=lambda _, loss: loss)
        
        return model

    def build_binary_classification_model(self):
        
        # Create binary classification model
        name_input = Input(shape=())
        timestamp_input = Input(shape=())
        message_input = Input(shape=())
        bag1_input = Input(shape=self.input_shape)
        bag2_input = Input(shape=self.input_shape)
        
        # Encode bag of contexts using the same encoder model
        encoded1 = self.encoder(bag1_input)
        encoded2 = self.encoder(bag2_input)
        for layer in self.encoder.layers:
            layer.trainable = False
            
        name_reshaped = Reshape((1,))(name_input)
        timestamp_reshaped = Reshape((1,))(timestamp_input)
        message_reshaped = Reshape((1,))(message_input)

        
        merged = Concatenate()([encoded1, encoded2, name_reshaped, timestamp_reshaped, message_reshaped])
        
        # Binary classification output
        binary_classification = Dense(units=512, activation=self.activation_fn3)(merged)
        binary_classification = Dense(units=512, activation=self.activation_fn3)(binary_classification)
        binary_classification = Dense(units=256, activation=self.activation_fn3)(binary_classification)
        binary_classification = Dense(units=1, activation=self.activation_fn3)(binary_classification)
        
        # Define model
        model = tf.keras.Model(inputs=[name_input, timestamp_input, message_input, bag1_input, bag2_input], outputs=binary_classification)
        
        # Compile model
        model.compile(optimizer=self.optimizer, loss="mse")
        
        return model

    def fit_siam(self, X_train, epochs, batch_size, verbose=0):        
        self.siam_model.fit([X_train, X_train], [X_train, X_train], epochs=epochs, batch_size=batch_size, verbose=verbose)
        
    def fit_binary_classification(self, X_train, y_train, epochs, batch_size, verbose=0):
        self.binary_classification_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)

    def evaluate_binary_classification(self, X_test, y_test, verbose=0):
        return self.binary_classification_model.evaluate(X_test, y_test, verbose=verbose)


In [73]:
model = CommitDiffModel()
#TODO: Modify ML model hyperparams

In [74]:
model.initialize()

### Model Unsupervised Training

In [75]:
model.fit_siam(np.array(ALL_BAGS_OF_CONTEXTS), epochs=8, batch_size=8, verbose=1)

Epoch 1/8
14/14 [==============================] - 3s 5ms/step - loss: -0.3427
Epoch 2/8
14/14 [==============================] - 0s 6ms/step - loss: -0.5207
Epoch 3/8
14/14 [==============================] - 0s 7ms/step - loss: -0.5782
Epoch 4/8
14/14 [==============================] - 0s 5ms/step - loss: -0.6293
Epoch 5/8
14/14 [==============================] - 0s 7ms/step - loss: -0.6510
Epoch 6/8
14/14 [==============================] - 0s 5ms/step - loss: -0.6610
Epoch 7/8
14/14 [==============================] - 0s 7ms/step - loss: -0.6726
Epoch 8/8
14/14 [==============================] - 0s 6ms/step - loss: -0.6835


### Model Supervised Training

In [76]:
model.fit_binary_classification(X_train, np.array(y_train), epochs=4, batch_size=32, verbose=1)

IndexError: tuple index out of range

In [ ]:
model.evaluate_binary_classification(X_test, np.array(y_test), verbose=0)

### Model Visualization

In [ ]:
plot_model(model.siam_model, show_shapes=True, show_layer_names=True)

In [ ]:
plot_model(model.encoder, show_shapes=True, show_layer_names=True)

In [ ]:
plot_model(model.binary_classification_model, show_shapes=True, show_layer_names=True)